In [14]:
! pip install sqlalchemy tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)


In [15]:
from sqlalchemy import create_engine, text
from tabulate import tabulate

### Database credentials

In [1]:
DB_USER = "myuser"
DB_PASS = "mypassword"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "mydb"

### Construct the database URL

In [2]:
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

In [5]:
engine = create_engine(DATABASE_URL)

### Test the connection

In [6]:
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT 1"))
        print("Connection successful:", result.scalar())
except Exception as e:
    print("Connection failed:", e)
    raise 

Connection successful: 1


# --- CREATE DATABASE OBJECTS ---

## 1. Create Table

In [21]:
try:
    with engine.connect() as connection:
        connection.execute(text("DROP TABLE IF EXISTS Customers;"))
        connection.commit()
        print("Table 'Customers' dropped (if it existed).")
except Exception as e:
    print("Error dropping table:", e)

Table 'Customers' dropped (if it existed).


In [22]:
try:
    with engine.connect() as connection:
        connection.execute(text("""
            CREATE TABLE Customers (
                CustomerID SERIAL PRIMARY KEY,
                FirstName VARCHAR(255),
                LastName VARCHAR(255),
                Email VARCHAR(255) UNIQUE
            );
        """))
        connection.commit()
        print("Table 'Customers' created successfully.")
except Exception as e:
    print("Error creating table:", e)

Table 'Customers' created successfully.


### Check 

In [23]:
try:
    with engine.connect() as connection:
        connection.execute(text("""
            INSERT INTO Customers (FirstName, LastName, Email) VALUES
            ('John', 'Doe', 'john.doe@example.com'),
            ('Jane', 'Smith', 'jane.smith@example.com');
        """))
        connection.commit()
        print("Data inserted into Customers table.")
except Exception as e:
    print("Error inserting data:", e)

Data inserted into Customers table.


In [24]:
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT * FROM Customers"))

        rows = result.fetchall()

        column_names = result.keys()

        table = tabulate(rows, headers=column_names, tablefmt="grid")

        print(table)

except Exception as e:
    print("Error querying table:", e)

+--------------+-------------+------------+------------------------+
|   customerid | firstname   | lastname   | email                  |
+==============+=============+============+========================+
|            1 | John        | Doe        | john.doe@example.com   |
+--------------+-------------+------------+------------------------+
|            2 | Jane        | Smith      | jane.smith@example.com |
+--------------+-------------+------------+------------------------+


## 2. Create Index

### Database Indexing

**What is it?**

An index is a special data structure that databases use to speed up data retrieval.

**Purpose:**

*   **Speed up queries:** Indexes drastically improve the performance of `SELECT` queries, especially those with `WHERE` clauses that filter data.
*   **Faster sorting & grouping:** Indexes can speed up `ORDER BY` and `GROUP BY` operations.


**Trade-offs:**

*   **Increased storage space:** Indexes consume disk space.
*   **Slower write operations:** `INSERT`, `UPDATE`, and `DELETE` operations can be slower because the index must be updated along with the table data.

**In short: Indexes make reads faster but writes slower, using more storage space. Use them wisely!**

In [36]:
try:
    with engine.connect() as connection:
        connection.execute(text("""
            DROP INDEX IF EXISTS idx_Email;
        """))
        
        connection.execute(text("""
            CREATE INDEX idx_Email ON Customers (Email);
        """))
        
        connection.commit()
        print("Index 'idx_Email' created successfully.")
        
except Exception as e:
    print("Error creating index:", e)

Index 'idx_Email' created successfully.


## 3. Create View

### Database Views

**What is it?**

A view is a virtual table based on the result-set of an SQL query. It doesn't store data physically; instead, it stores the *query* that defines how the data is derived from one or more underlying tables.

**Purpose:**

*   **Simplified queries:** Views hide the complexity of underlying table structures and complex joins, presenting a simplified and focused data representation to users.
*   **Data security:** Views can restrict access to certain columns or rows of a table, providing a security layer by exposing only the necessary data to specific users or applications.
*   **Data consistency:** Views ensure that all users see the same data based on the defined query, maintaining consistency across different applications.
*   **Data abstraction:**  Views decouple applications from the underlying table structure. If the table structure changes, you can often update the view's query without affecting the applications that use the view.
*   **Improved readability:** Complex queries can be encapsulated in views, making the code more readable and maintainable.

**Key Characteristics:**

*   **Virtual:** Views don't store data physically.
*   **Dynamic:** Data in a view is always up-to-date, reflecting the current data in the underlying tables.
*   **Read-Only (typically):**  While some databases allow updating views under certain conditions, views are generally considered read-only. You usually update the underlying tables directly, and the view reflects those changes.

**In short: Views are pre-defined queries that simplify data access, provide security, and improve code readability.**

In [35]:
try:
    with engine.connect() as connection:
        connection.execute(text("""
            DROP VIEW IF EXISTS CustomerView;
        """))
         
        connection.execute(text("""
            CREATE VIEW CustomerView AS
            SELECT CustomerID, FirstName, LastName
            FROM Customers;
        """))
        connection.commit()
        print("View 'CustomerView' created successfully.")
except Exception as e:
    print("Error creating view:", e)

View 'CustomerView' created successfully.


In [37]:
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT * FROM CustomerView"))
        rows = result.fetchall()
        column_names = result.keys()
        table = tabulate(rows, headers=column_names, tablefmt="grid")
        print(table)

except Exception as e:
    print("Error querying CustomerView:", e)

+--------------+-------------+------------+
|   customerid | firstname   | lastname   |
+==============+=============+============+
|            1 | John        | Doe        |
+--------------+-------------+------------+
|            2 | Jane        | Smith      |
+--------------+-------------+------------+
